In [1]:
%matplotlib inline

In [2]:
import pandas as pd

# We should consider filtering our data a bit because it's a bit messy currently

**I've written this scaffold to make things easier but code similar to what I've written below should be written as a function or two and run for every host and every virus probably saving as a new "tsv" file called "yadayada.clean.tsv"**

**This currently works on E. coli but it's possible that there will be some weird genomes that something in here screws up on so we'll need to think about double checking things to make sure that we never remove too much data. Which is to say that df.shape shouldn't change *that* dramatically from beginning to end and if it does it's likely that there is a bug in the code that needs to be investigated**

In [16]:
###Read in the starting TSV
df = pd.read_csv('../Data/host_genomes/562.tsv', sep='\t', index_col=0)
###Reset the index so that it runs from zero to whatever
df = df.reset_index(drop=True)
print(df.shape)
df.head()

(4379, 12)


,genome_id,source,type,start,stop,idk,strand,trash,qualifiers,coding_sequence,upstream_sequence,energy_binding
0,NC_000913.3,RefSeq,CDS,190,255,.,+,0,ID=cds-NP_414542.1;Parent=gene-b0001;Dbxref=Un...,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,TTACAGAGTACACAACATCC,-2.4
1,NC_000913.3,RefSeq,CDS,337,2799,.,+,0,ID=cds-NP_414543.1;Parent=gene-b0002;Dbxref=Un...,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,AAGGTAACGAGGTAACAACC,-5.4
2,NC_000913.3,RefSeq,CDS,2801,3733,.,+,0,ID=cds-NP_414544.1;Parent=gene-b0003;Dbxref=Un...,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,ATGGAAGTTAGGAGTCTGAC,-6.3
3,NC_000913.3,RefSeq,CDS,3734,5020,.,+,0,ID=cds-NP_414545.1;Parent=gene-b0004;Dbxref=Un...,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,CACGAGTACTGGAAAACTAA,-3.3
4,NC_000913.3,RefSeq,CDS,5234,5530,.,+,0,ID=cds-NP_414546.1;Parent=gene-b0005;Dbxref=Un...,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,AATGATAAAAGGAGTAACCT,-6.3


**First, make sure the table only uses each `locus_tag` once. These should be unique identifiers for the hosts and should therefore only appear once. If they appear more than once I have no idea what's happening. The virus dataframes won't be able to run this part of the code, however.**

In [17]:
df['locus_tag'] = df['qualifiers'].str.split('Parent=gene-', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]

In [18]:
df['locus_tag'].value_counts()[:10]

b0470    3
b4492    3
b0484    3
b4623    3
b3046    2
b0240    2
b4569    2
b2891    2
b3443    2
b4579    2
Name: locus_tag, dtype: int64

**Oooof, let's get rid of those**

In [19]:
df = df.drop_duplicates(subset=['locus_tag'], keep=False)
print(df.shape)

(4335, 13)


In [20]:
df['locus_tag'].value_counts()[:10]

b2023    1
b3785    1
b2424    1
b2734    1
b2625    1
b2825    1
b2137    1
b2075    1
b3655    1
b2948    1
Name: locus_tag, dtype: int64

**Voila!**

**How about any weird length genes? This should be checked and removed for all genomes (host and virus)**

In [21]:
((df['stop']-df['start']) %3).value_counts()

2    4305
1      22
0       8
dtype: int64

**Clean them up too**

In [22]:
df = df[(df['stop']-df['start']) %3 == 2]
print(df.shape)

(4305, 13)


**Remove phage/prophage genes from descriptors. This again should only be run for the host genomes so you can see maybe we'll want separate functions like "def clean_host(...)" and "def clean_virus(...)"**

In [23]:
filter_word = 'prophage'
# filter_word = 'phage'
virus_genes = df[(df['qualifiers'].str.contains(filter_word)==True)]
print(virus_genes.shape)

(205, 13)


In [24]:
df = df[(df['qualifiers'].str.contains(filter_word)==False)]
df.shape

(4100, 13)

# Add in knowledge of codon usage bias

** This is an important control and you'll have to install this "iCUB" software to make this run. I worked on this project with a former graduate student in my PhD lab and the code is here: https://github.com/amarallab/iCUB **

**The major bug that I can anticipate cropping up is if the organism uses a "non-standard" genetic code but we'll cross that bridge when we get there. This will also probably cause an error if any "non-standard" nucleotides appear in a given genome which will almost certainly occur and need to be fixed**

In [3]:
import sys
sys.path.append('/Users/ChaseWeaver/Desktop/Projects/iCUB-master/')
import iCUB

In [14]:
for index in df.index:
    nt_seq = df.at[index, "coding_sequence"]
    if len(nt_seq)%3 == 0:
        df.at[index, "iCUB"] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()

**If our genes are clean and good then we shouldn't have any null values in either of these categories and we can then write this new dataframe**

In [15]:
print(df[df['iCUB'].isnull()==True].shape)
print(df[df['energy_binding'].isnull()==True].shape)

(0, 14)
(0, 14)


# Final notes:

**As I mentioned, we aren't going to want to step through this code for every genome so your task is to write this up into some functions that you can then put into for loops and run for everyone. I'm imagining something like:**

<code>
for host_tsv_file in glob.glob('blahblah/\*.tsv): 
    clean_host_tsv(host_tsv_file)
</code>
    
**where `clean_host_tsv()` will read the tsv, clean it up, check that no problems occurred, and write the new tsv. Something along the lines of:**

<code>
def clean_host_tsv(tsv_location):

    df = pd.read_tsv(tsv_location...)
    starting_shape = df.shape
    #Code that does the work goes here
    ending_shape = df.shape
    #Code that compares ending and starting shapes to make sure things look reasonable goes here
    df.to_csv(tsv_location.replace('.tsv', '.clean.tsv'))
</code>


**And then again we'll have something like this for the viruses:**

<code>
for virus_folder in glob.glob('blahblah/\*\_rep\_viruses/'):
    for virus_tsv_file in glob.glob(virus_folder + '\*.tsv'):
        clean_virus_tsv(virus_tsv_file)
</code>


In [16]:
df = pd.read_csv('../Data/host_genomes/287.tsv', sep = ",", index_col = 0)
print(df.shape)

(5573, 12)


In [18]:
def clean_host_tsv(tsv_location):
    
    df = pd.read_csv(tsv_location, sep = ",", index_col = 0)
    initial_shape = df.shape
    df = df.reset_index(drop = True)
    
    df['locus_tag'] = df['qualifiers'].str.split('Parent=gene-', n=1, expand=True)[1]\
                            .str.split(';', n=1, expand=True)[0]
    df = df.drop_duplicates(subset = ["locus_tag"], keep = False)
    df = df[(df['stop']-df['start']) %3 == 2]
    filter_word = 'prophage'
    virus_genes = df[(df['qualifiers'].str.contains(filter_word)==True)]
    df = df[(df['qualifiers'].str.contains(filter_word)==False)]
    
    for index in df.index:
        nt_seq = df.at[index, "coding_sequence"]
        if len(nt_seq)%3 == 0:
            df.at[index, "iCUB"] = iCUB.iCUB_Calculator(nt_seq).get_iCUB()
    
    return df.shape
    

In [19]:
clean_host_tsv("../Data/host_genomes/287.tsv")

(5571, 14)

In [15]:
print(df.shape)

(5573, 12)
